# Deep Q-learning

Recall: the state-value is the expected cumulative reward $v_{\pi}(s_t) = \mathbb{E}_\pi[\sum_{k=0} w_k R_{t+k+1}]$, where $w_k$ are weights and $R_i$ are future rewards starting from $s_t$ ($R_{t+1}$ is the reward observed by taking
the action $a_t$ from $s_t$) and following the policy $\pi$ (the expectation is taken over all possible _trajectories_ starting from $s_t$). In general, the weights should discount future rewards
compared to recent ones, i.e. $w_k = \gamma^k$, with  $0 \leq \gamma \leq 1$ the _discount factor_.

Similarly, the action-value function $Q_\pi(s,a)$ takes a state and an action as inputs and returns the value of taking that action given that state and
following $\pi$. A Q function can be represented by a _Q-table_, where each entry corresponds to a state-action pair value. This table can be _very_ large if the state space is very
large. 

Notice that when we have an optimal Q function, we have an optimal policy since we know the best action to take at each state: $\pi(s) = \argmax_a Q(s,a)$.

The problem is that to calculate EACH value of a state or a state-action pair, we need
to sum all the rewards an agent can get if it starts at that state. The **Bellmann
Equation** simplifies the calculation of the value function:

$$v_\pi(s_t) = \mathbb{E}_\pi[R_{t+1} + \gamma v_\pi(s_{t+1})].$$

Analogously, for the _optimal action-value function_ $q_\star(s,a) = \textrm{max}_\pi\,
Q_\pi(s,a)$ it holds

$$q_\star(s_t,a_t) = \mathbb{E}[R_{t+1} + \gamma\,\textrm{max}_{a}\,q_\star(s_{t+1},a)].$$

**Q-learning**: algorithm that learns to predict the value of a
state-action pair (Q function); the prediction is compared to the observed accumulated rewards at
some later time to update the parameters of the algorithm. 
Based on the Bellmann equation for $q_\star$, Q-learning algorithms use the update rule ($\alpha$ is
the _learning rate_ and $Q$ is the optimal action-value function)

$$Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \alpha[R_{t+1}+ \gamma \max Q(s_{t+1},a)-Q(s_t,a_t)] $$

i.e., the $Q$ value at time $t$ is updated to be the current predicted $Q$ value plus the
(discounted) amount of value we expect in the future (state $s_{t+1}$), given that we
play optimally from our current state. 

<figure>
    <img src="Qlearning_algo.png" alt="Caption" width="800" />
    <figcaption>Q-learning algorithm (taken from huggingface.co).</figcaption>
</figure>

## Solving the Gridworld environment with (Deep) Q-learning

_Gridworld_ is a game where the player moves on a square board (or grid). The player has
to reach the goal (+), avoiding the wall (W) and the pit (-). Every nonwinning move
receives -1 as a reward. The winning move (reaching the goal) receives a +10, while
there is a -10 reward for losing the game (landing on the pit).

In [ ]:
from Gridworld import Gridworld
import numpy as np
import jax.numpy as jnp
import random
from matplotlib import pylab as plt
from flax import nnx
import optax
# static = start from predetermined configuration (for player and objects)
# player = just the player is initialized at random location
# random = both player and objects are randomly placed
game = Gridworld(size=4, mode='static')

In [ ]:
# display current state
game.display()

In [ ]:
# example moves
game.makeMove('d')
game.makeMove('d')
game.makeMove('l')
game.display()

In [ ]:
game.reward()

In [ ]:
# representation of the state as a 4x4x4 tensor, i.e. one-hot encoding of (player, goal,
# pit, wall) locations
game.board.render_np()

In [ ]:
action_set = {
    0: 'u',
    1: 'd',
    2: 'l',
    3: 'r',
}

1. Build a neural network to represent Q function. More precisely, the network represents a
vector-valued Q function that accepts a state and returns a _vector_ of state-action
values, one for each possible action. Suggested architecture: 2 hidden layers (150 and
100 units, respectively), with ReLU activations. The input to the network should be the
state represented as a 1x64 array. 

In [ ]:
class Model(nnx.Module):
  def __init__(self, l1, l2, l3, l4, rngs: nnx.Rngs):
    # ...
    pass

  def __call__(self, x):
    # y = ...
    return y

l1 = 64 # input: state as a 1x64 vector
l2 = 150
l3 = 100
l4 = 4 # outputs

model = Model(l1, l2, l3, l4, nnx.Rngs(0))

2. Define the function that computes the *loss function* for training the Q-network,
   i.e. the mean squared error between the Q-value predicted by the network for the
   current action and the _target_ given by the update rule above (assuming $\alpha=1$
   and $\gamma=0.9$). Take actions according to a $\epsilon$-greedy policy.

In [ ]:
learning_rate = 1e-3
optimizer = nnx.ModelAndOptimizer(model, optax.adam(learning_rate))

gamma = 0.9
epsilon = 1.0

def loss_fn(model, state):
    # compute the Q-values for the current state
    # qval = ...

    if (random.random() < epsilon):
        action_ = np.random.randint(0,4)
    else:
        action_ = int(np.argmax(qval))
    
    action = action_set[action_]
    # perform the action in the environment
    # ...

    # get the new state and reward
    # state2 = ...
    # reward = ...

    # compute the Q-values for the new state
    # newQ = ...

    # compute the maximum Q-value for the new state
    # maxQ = ...

    if reward == -1:
        # compute the target Q-value using the Bellman equation
        # Y = ...
    else:
        # assign immediate reward if goal or pit reached
        Y = reward

    Y = jnp.asarray([Y])
    X = jnp.array([qval.squeeze()[action_]])

    return optax.losses.squared_error(X, Y).mean(), (reward, state2)

2. Define the function that performs a *training epoch*, i.e. updates the Q-network's
   weights based on the gradient of the loss. 

In [ ]:
def _train_epoch(model, optimizer, state, game):
    # ...
    return loss, reward, state2

3. Train the Q-network by playing games.

In [ ]:
games = 1000
losses = []
for i in range(games):
    game = Gridworld(size=4, mode='static')
    # add some noise to the initial state (because most of the state elements are 0,
    # potential problem with ReLU activations)
    state1 = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
    status = 1
    while(status == 1):
        loss, reward, state1 = _train_epoch(model, optimizer, state1, game)
        losses.append(loss.item())
        if reward != -1:
            status = 0
    # reduce epsilon as learning progresses
    if epsilon > 0.1:
        epsilon -= (1/games)

In [ ]:
plt.plot(losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

In [ ]:
# Helper function to play games with trained models

def test_model(model, mode='static', display=True):
    i = 0
    test_game = Gridworld(mode=mode)
    state = test_game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
    if display:
        print("Initial State:")
        print(test_game.display())
    status = 1
    while(status == 1):
        qval = model(state)
        action_ = int(np.argmax(qval))
        action = action_set[action_]
        if display:
            print('Move #: %s; Taking action: %s' % (i, action))
        test_game.makeMove(action)
        state = test_game.board.render_np().reshape(1,64) + np.random.rand(1,64)/10.0
        if display:
            print(test_game.display())
        reward = test_game.reward()
        if reward != -1:
            if reward > 0:
                status = 2
                if display:
                    print("Game won! Reward: %s" % (reward,))
            else:
                status = 0
                if display:
                    print("Game LOST. Reward: %s" % (reward,))
        i += 1
        if (i > 15):
            if display:
                print("Game lost; too many moves.")
            break
    
    win = True if status == 2 else False
    return win

4. Test the model starting from the training initial condition and from a random one
   (repeat multiple times and notice _catastrophic forgetting_)

In [ ]:
test_model(model, 'static')

In [ ]:
test_model(model, 'random')

### Introducing experience replay

We can use **experience replay** to counter _catastrophic forgetting_ (i.e. similar
games with very different outcomes, leading to inability to properly learn anything).
Here is the algorithm:
1. In state $s_t$ take action $a_t$ and observe the new state $s_{t+1}$ and reward $R_{t+1}$.
2. Store the tuple $(s_t, a_t, s_{t+1}, R_{t+1})$ in the replay buffer.
3. Continue to store each experience in the buffer until you have filled it to a
   specific length.
4. Once the experience replay memory is filled, randomly select a batch (with a given
   batch size).
5. Calculate updates of the $Q$ value for each element of the batch and store them in a
   target array ($Y$).
6. Use the Q-value predicted by the network for each state-action in the batch and the
   updates $Y$ for training (updating the parameters of the network). For subsequent
   moves where the replay memory is full, overwrite old values.

Use the following class for the replay buffer to implement Q-learning with experience replay.

In [ ]:
import random
from collections import deque
import numpy as np

class ReplayBuffer:
    def __init__(self, max_size):
        # a deque is a list with a maximum size, so that if you try to append something
        # when it is already full, it will overwrite the first item in the list
        self.buffer = deque(maxlen=max_size)

    def add(self, experience):
        self.buffer.append(experience)

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        return map(np.array, zip(*batch))

    def size(self):
        return len(self.buffer)

In [ ]:
# Use the same model and optimizer definitions as before
l1 = 64
l2 = 150
l3 = 100
l4 = 4

model = Model(l1, l2, l3, l4, nnx.Rngs(0))

learning_rate = 1e-3
optimizer = nnx.ModelAndOptimizer(model, optax.adam(learning_rate))

1. Implement the loss function and the Q-learning update function (look at the `train`
   function below first to understand what the arguments and return values of the update
   function are). Make them suitable to be jitted using `nnx.jit` (i.e. only use
   jax-compatible operations).

In [ ]:
@nnx.jit
def loss_fn(model, states, q_updates, actions):
    # compute Q-values for all actions, for all states in the batch
    # q_values = ...
    # use jnp.take_along_axis to select the Q-values corresponding to the taken actions
    # q_value = ...
    loss = optax.losses.squared_error(q_value, q_updates).mean()
    return loss

@nnx.jit
def q_learning_update(model, states, actions, rewards, next_states, dones, optimizer):
    # compute Q-values for next states
    # next_q_values = ...
    # compute max Q-value for next states (reshape to be a column vector)
    # max_next_q_values = ...

    q_updates = rewards + gamma * max_next_q_values * (1 - dones)

    # compute loss and update weights
    # ...
    
    return loss

2. Train the Q-network with experience replay by playing games.

In [ ]:
replay_buffer_size = 1000
batch_size = 200
games = 5000 
epsilon = 0.3
gamma = 0.9
losses = []
total_rewards = []
max_moves = 50

def train():
    replay_buffer = ReplayBuffer(replay_buffer_size)

    for i in range(games):
        game = Gridworld(size=4, mode='random')
        state = game.board.render_np().flatten() + np.random.rand(64)/100.0
        mov = 0
        done = False
        total_reward = 0

        while not done:
            mov += 1
            if (random.random() < epsilon):
                action_ = np.random.randint(0,4)
            else:
                qval = model(state)
                action_ = int(np.argmax(qval))
            
            action = action_set[action_]
            game.makeMove(action)
            next_state = game.board.render_np().flatten() + np.random.rand(64)/100.0
            reward = game.reward()

            if reward != -1 or mov > max_moves:
                done = True

            replay_buffer.add((state, np.array([action_]), np.array([reward]), next_state, np.array([done])))

            if replay_buffer.size() >= batch_size:
                batch = replay_buffer.sample(batch_size)
                states, actions, rewards, next_states, dones = batch
                # states: (batch_size, 64)
                # actions: (batch_size, 1)
                # rewards: (batch_size, 1)
                # next_states: (batch_size, 64)
                # dones: (batch_size, 1)
                loss = q_learning_update(model, states, actions, rewards, next_states, dones, optimizer)
                losses.append(loss)

            state = next_state
            total_reward += reward


        total_rewards.append(total_reward)

        if i%100 == 0:
            print(f"Game: {i}, Total Reward: {total_reward}")

In [ ]:
train()

In [ ]:
losses = np.array(losses)
total_rewards = np.array(total_rewards)

In [ ]:
def running_mean(x,N=50):
    c = x.shape[0] - N
    y = np.zeros(c)
    conv = np.ones(N)
    for i in range(c):
        y[i] = (x[i:i+N] @ conv)/N
    return y


In [ ]:
plt.plot(losses)
plt.xlabel("Episodes")
plt.ylabel("Loss")
plt.show()

In [ ]:
plt.plot(running_mean(total_rewards))
plt.xlabel("Episodes")
plt.ylabel("Total reward")
plt.show()

3. Test the model with a random initial condition using the `test_model` helper function.

4. Evaluate the percentage of won games out of 1000 using the trained model.

Extensions:

- Grid size: increase the grid size to make the task more challenging.
- Action space: extend the action space (e.g., diagonal moves).
- Multiple obstacles: add more obstacles to the environment.